In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess


class ESSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "ESS"
    
    # URL(s) to start with.
    start_urls = [
        'http://www.everywhereist.com/blog/',
    ]

    # Use XPath to parse the response we get.
    def parse(self, response):
        
        # Iterate over every <article> element on the page.
        for article in response.xpath('//article'):
            
            # Yield a dictionary with the values we want.
            yield {
                # This is the code to choose what we want to extract
                # You can modify this with other Xpath expressions to extract other information from the site
                'name': article.xpath('string(header/h2/a)').extract(), #works
                'date': article.xpath('string(header/div[@class="date"]/div[@class="actual-date"])').extract(),
                'text': article.xpath('div[@class="body"]/p/text()').extract(), #works
                #'tags': article.xpath('*/span[@class="tag-links"]/a/text()').extract()
            }

# Tell the script how to run the crawler by passing in settings.
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'secondpage.json',  # Name our storage file.
    'LOG_ENABLED': False           # Turn off logging for now.
})

# Start the crawler with our spider.
process.crawl(ESSpider)
process.start()
print('Success!')

Success!


In [2]:
import pandas as pd

secondpage = pd.read_json('secondpage.json', orient='records')
print(secondpage.shape)
print(secondpage.head())

(9, 3)
             date                                               name  \
0   [Sep 4, 2018]                      [I Quit Twitter for a Month.]   
1  [Jul 30, 2018]  [What Happened When I Tried Talking to Twitter...   
2  [Jul 19, 2018]                                [Lost And Founder.]   
3  [Jun 22, 2018]                          [Scenes From a Bookstore]   
4  [Jun 21, 2018]                   [782 Words About Writer’s Block]   

                                                text  
0  [I quit Twitter for a month. This should not s...  
1  [Trigger warning: please note that this is a b...  
2  [My beloved wrote a book. How it came to be wa...  
3  [“Why do they have so many copies of  my book?...  
4  [I don’t know what writer’s block looks like f...  


In [3]:
secondpage.loc[0,'name']

['I Quit Twitter for a Month.']